## Imports

In [45]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## Data Read-in

In [46]:
df = pd.read_csv('redfin_2023-10-05-09-31-42.csv')
BC_gf = gpd.read_file('countyboundary')

## Data Clean

In [47]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [48]:
df = df.dropna(subset=['SOLD DATE'])

In [49]:
# Define list of desired months (excluding current month)
desired_months = ['September']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [50]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    820
Name: count, dtype: int64
-------
$/SQUARE FEET
False    803
True      17
Name: count, dtype: int64
-------
YEAR BUILT
False    820
Name: count, dtype: int64
-------


In [51]:
# sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
# second_newest_building = sorted_df.iloc[2]
# print(second_newest_building['URL'])

In [52]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [53]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [54]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
23,PAST SALE,September-20-2023,Condo/Co-op,365 W Laurel Dr Unit N4,Margate,FL,33063.0,115.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/365-W-Laurel...,Beaches MLS,F10388213,N,Y,26.233121,-80.196086
559,PAST SALE,September-25-2023,Condo/Co-op,940 Hillcrest Ct #111,Hollywood,FL,33021.0,180.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/940-Hillcr...,MARMLS,A11391582,N,Y,26.001763,-80.193696
145,PAST SALE,September-26-2023,Condo/Co-op,2271 NW 47th Ter #106,Lauderhill,FL,33313.0,63000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/2271-NW-4...,Beaches MLS,F10386966,N,Y,26.155254,-80.213700
170,PAST SALE,September-22-2023,Condo/Co-op,3551 Inverrary Dr #203,Lauderhill,FL,33319.0,70000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/3551-Inve...,Beaches MLS,F10394081,N,Y,26.172338,-80.232436
57,PAST SALE,September-14-2023,Condo/Co-op,200 Ashbury Rd #107,Hollywood,FL,33024.0,77500.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/200-Ashbur...,MARMLS,A11440794,N,Y,26.044667,-80.228058
500,PAST SALE,September-22-2023,Condo/Co-op,3150 Holiday Springs Blvd Unit 8-206,Margate,FL,33063.0,79000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3150-Holiday...,MARMLS,A11420190,N,Y,26.265619,-80.233842
393,PAST SALE,September-15-2023,Condo/Co-op,4411 NW 16th St #204,Lauderhill,FL,33313.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4411-NW-1...,Beaches MLS,F10392031,N,Y,26.147452,-80.209168
419,PAST SALE,September-20-2023,Condo/Co-op,4801 NW 22nd Ct #202,Lauderhill,FL,33313.0,80000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/4801-NW-2...,MARMLS,A11303009,N,Y,26.155284,-80.214136
356,PAST SALE,September-26-2023,Condo/Co-op,3241 Holiday Springs Blvd #308,Margate,FL,33063.0,84000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/3241-Holiday...,Beaches MLS,F10384080,N,Y,26.268702,-80.230955
194,PAST SALE,September-26-2023,Condo/Co-op,2750 W Sunrise Lakes Dr #307,Sunrise,FL,33322.0,85000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/2750-W-Sunri...,Beaches MLS,F10396232,N,Y,26.159588,-80.255872


In [55]:
print(df_filtered['URL'].iloc[559])

https://www.redfin.com/FL/Hollywood/940-Hillcrest-Ct-33021/unit-111/home/41942179


In [56]:
# Correct the prices, if needed
df_filtered.at[23,'PRICE']=(115000)
df_filtered.at[559,'PRICE']=(180000)

In [57]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [58]:
# Corrections, if needed
df_filtered.at[23,'$/SQUARE FEET']=(1150000/1019)
df_filtered.at[559,'$/SQUARE FEET']=(180000/961)

In [59]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
145,63000.0,2271 NW 47th Ter #106,Lauderhill,68.0
419,80000.0,4801 NW 22nd Ct #202,Lauderhill,86.0
170,70000.0,3551 Inverrary Dr #203,Lauderhill,96.0
607,125000.0,8240 SW 24th St #5315,North Lauderdale,96.0
356,84000.0,3241 Holiday Springs Blvd #308,Margate,97.0
819,104900.0,3300 Spanish Moss Ter #206,Lauderhill,105.0
153,110000.0,3090 NW 46th Ave Unit 107A,Lauderdale Lakes,108.0
222,104000.0,5000 NW 36th St #402,Lauderdale Lakes,110.0
221,96000.0,1101 NW 58th Ter #207,Sunrise,110.0
500,79000.0,3150 Holiday Springs Blvd Unit 8-206,Margate,110.0


In [60]:
print(df_filtered.URL.iloc[8])

https://www.redfin.com/FL/Pembroke-Pines/711-SW-111th-Way-33025/unit-304/home/42065267


In [61]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [62]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [63]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [64]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [65]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [66]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [67]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"September 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(BC_gf,tooltip='Broward County',name='Broward County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [68]:
m.save('index.html')

## Summary Info

In [69]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [70]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [71]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [83]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/2840-N-Ocean-Blvd-33308/unit-903/home/41715817


In [73]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
2711 HOLLYWOOD BEACH COND, 2711 S Ocean Dr Unit 402L Hollywood | Price $3,725,000 | $943 psf | Year built: 2009
Least Expensive
Castle Apartments, 2271 NW 47th Ter #106 Lauderhill | Price $63,000 | $68 psf | Year built: 1972
Highest Price Per Square Foot
Sabbia Beach, 730 N Ocean Blvd #503 Pompano Beach | Price $2,495,000 | $1,327 psf | Year built: 2020
Lowest Price Per Square Foot
Castle Apartments, 2271 NW 47th Ter #106 Lauderhill | Price $63,000 | $68 psf | Year built: 1972
Newest
Sabbia Beach, 730 N Ocean Blvd #503 Pompano Beach | Price $2,495,000 | $1,327 psf | Year built: 2020
Oldest
Sea Tower, 2840 N Ocean Blvd #903 Fort Lauderdale | Price $510,000 | $663 psf | Year built: 1958


## Time on Market Calculator

In [74]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Hollywood/2711-S-Ocean-Dr-33019/unit-402L/home/42081716


In [82]:
from datetime import datetime, timedelta

date1 = datetime(2023, 6, 26) ## List (Earlier) date
date2 = datetime(2023, 9, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

92


## Map URL Snagger

In [76]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [77]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_condo_sales_month_ending_sep_2023


## Get Summary Data

In [78]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 820
--------
Total sale price: $251,146,176
--------
Median sale price: $235,000
--------
Max sale price: $3,725,000
--------
Min sale price: $63,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,327
--------
Min price per square foot: $68
--------
Median price per square foot: $233
------------------------------------------------
CONDO AGES
Newest building: 2020.0
----------
Oldest building: 1958.0
----------
Average building age: 1980.9475609756098
